# Data loading pipeline examples

The purpose of this notebook is to illustrate reading Nifti files and test speed of different methods.

In [ ]:
%matplotlib inline

import os
import sys
from glob import glob
import tempfile

import numpy as np
import nibabel as nib


import torch
from torch.utils.data import DataLoader
from torch.multiprocessing import Pool, Process, set_start_method
try:
     set_start_method('spawn')
except RuntimeError:
    pass

import monai
from monai.data import ArrayDataset
from monai.transforms import (Compose, LoadNifti, AddChannel, ScaleIntensity,
                              ToTensor, RandSpatialCrop, Rotate, RandAffine)

monai.config.print_config()

### 0. Preparing input data (nifti images)

Create a number of test Nifti files, 3d single channel images with spatial size (256, 256, 256) voxels.

In [ ]:
tempdir = tempfile.mkdtemp()

for i in range(5):
    im, seg = monai.data.synthetic.create_test_image_3d(256,256,256)
    
    n = nib.Nifti1Image(im, np.eye(4))
    nib.save(n, os.path.join(tempdir, 'im%i.nii.gz' % i))
    
    n = nib.Nifti1Image(seg, np.eye(4))
    nib.save(n, os.path.join(tempdir, 'seg%i.nii.gz' % i))

In [ ]:
# prepare list of image names and segmentation names
images = sorted(glob(os.path.join(tempdir, 'im*.nii.gz')))
segs = sorted(glob(os.path.join(tempdir, 'seg*.nii.gz')))

### 1. Test image loading with minimal preprocessing

In [ ]:
imtrans = Compose([
    LoadNifti(image_only=True),
    AddChannel(),
    ToTensor()
])    

segtrans = Compose([
    LoadNifti(image_only=True),
    AddChannel(),
    ToTensor()
])    
    
ds = ArrayDataset(images, imtrans, segs, segtrans)
loader = DataLoader(ds, batch_size=3, num_workers=8)

im, seg = monai.utils.misc.first(loader)
print(im.shape, seg.shape)

In [ ]:
%timeit data = next(iter(loader))

### 2. Test image-patch loading with CPU multi-processing:

- rotate (256, 256, 256)-voxel in the plane axes=(1, 2)
- extract random (64, 64, 64) patches
- implemented in MONAI using ` scipy.ndimage.rotate`

In [ ]:
images = sorted(glob(os.path.join(tempdir, 'im*.nii.gz')))
segs = sorted(glob(os.path.join(tempdir, 'seg*.nii.gz')))

imtrans = Compose([
    LoadNifti(image_only=True),
    ScaleIntensity(),
    AddChannel(),
    Rotate(angle=45.),
    RandSpatialCrop((64, 64, 64), random_size=False),
    ToTensor()
])    

segtrans = Compose([
    LoadNifti(image_only=True),
    AddChannel(),
    Rotate(angle=45.),
    RandSpatialCrop((64, 64, 64), random_size=False),
    ToTensor()
])    
    
ds = ArrayDataset(images, imtrans, segs, segtrans)
loader = DataLoader(ds, batch_size=3, num_workers=8, pin_memory=torch.cuda.is_available())

im, seg = monai.utils.misc.first(loader)
print(im.shape, seg.shape)

In [ ]:
%timeit -n 3 data = next(iter(loader))

(the above results were based on Intel(R) Xeon(R) CPU E5-2650 v4 @ 2.20GHz)

### 3. Test image-patch loading with preprocessing on GPU:

- random rotate (256, 256, 256)-voxel in the plane axes=(1, 2)
- extract random (64, 64, 64) patches
- implemented in MONAI using native pytorch resampling

In [ ]:
images = sorted(glob(os.path.join(tempdir, 'im*.nii.gz')))
segs = sorted(glob(os.path.join(tempdir, 'seg*.nii.gz')))

# same parameter with different interpolation mode for image and segmentation
rand_affine_img = RandAffine(prob=1.0, rotate_range=np.pi/4, translate_range=(96, 96, 96),
                             spatial_size=(64, 64, 64), mode='bilinear',
                             as_tensor_output=True, device=torch.device('cuda:0'))
rand_affine_seg = RandAffine(prob=1.0, rotate_range=np.pi/4, translate_range=(96, 96, 96),
                             spatial_size=(64, 64, 64), mode='nearest',
                             as_tensor_output=True, device=torch.device('cuda:0'))
    
imtrans = Compose([
    LoadNifti(image_only=True),
    ScaleIntensity(),
    AddChannel(),
    rand_affine_img,
    ToTensor()
])    

segtrans = Compose([
    LoadNifti(image_only=True),
    AddChannel(),
    rand_affine_seg,
    ToTensor()
])    
    
ds = ArrayDataset(images, imtrans, segs, segtrans)
loader = DataLoader(ds, batch_size=3, num_workers=0)

im, seg = monai.utils.misc.first(loader)

print(im.shape, seg.shape)

In [ ]:
%timeit -n 3 data = next(iter(loader))

In [ ]:
print(torch.cuda.get_device_name(0))
print(torch.cuda.memory_summary(0, abbreviated=True))

In [ ]:
!rm -rf {tempdir}